In [1]:
import glob
import pandas as pd
import numpy as np

input_folder = "input/"
output_folder = "output/"
csv_separator = ","

all_filenames = glob.glob(input_folder + "*.csv")


In [2]:
for filename in all_filenames:
    print("nome do arquivo:" + filename)

    sample_csv = pd.read_csv(filename , sep = csv_separator)
    
    #create column Variant Frequency 1 with the value from first position of patter xyz% -> abc%  
    sample_csv['Variant Frequency 1'] = sample_csv['Variant Frequency'].str.split(' -> ').str[0]
    #removes the % and convert to numeric
    sample_csv['Variant Frequency 1'] = pd.to_numeric(sample_csv['Variant Frequency 1'].str.split('%').str[0])

    #to the same as above, but for the second column
    sample_csv['Variant Frequency 2'] = sample_csv['Variant Frequency'].str.split(' -> ').str[1]
    sample_csv['Variant Frequency 2'] = sample_csv['Variant Frequency 2'].fillna('0')
    sample_csv['Variant Frequency 2'] = pd.to_numeric(sample_csv['Variant Frequency 2'].str.split('%').str[0])

    # filter data
    sample_csv = sample_csv[sample_csv['gene'].notna()]
    sample_csv = sample_csv[sample_csv['Protein Effect'] == "Substitution"]
    sample_csv = sample_csv[sample_csv['Variant Frequency 1'] < 80.0 ] 
    
    # calculate aa position (CDS Position / 3, rounded up)
    sample_csv['AA Position'] = sample_csv['CDS Position'] / 3
    sample_csv['AA Position'] = sample_csv['AA Position'].apply(np.ceil).astype(int)
    
    sample_csv = sample_csv.drop(columns=['Name', 'CDS', 'Change','Codon Change', 'Coverage', 'Polymorphism Type'])

    # create output file
    output_filename = output_folder + filename.split("/")[1].split(".")[0] + "_processada.csv"
    sample_csv.to_csv(output_filename, sep=';', encoding='utf-8')
    

nome do arquivo:input/71890.csv
nome do arquivo:input/71262.csv
nome do arquivo:input/71974.csv
nome do arquivo:input/71331.csv
nome do arquivo:input/72025.csv
nome do arquivo:input/71718.csv
nome do arquivo:input/71213.csv
nome do arquivo:input/71189.csv
nome do arquivo:input/71950.csv
nome do arquivo:input/71029.csv


In [3]:
sample_csv.tail(100)

,gene,Amino Acid Change,CDS Position,Protein Effect,Variant Frequency,Unnamed: 11,Unnamed: 12,Unnamed: 13,Variant Frequency 1,Variant Frequency 2,AA Position
43,ORF1ab,F -> Y,15257,Substitution,25.0%,NaN,NaN,NaN,25.0,0.0,5086
46,ORF1ab,L -> K,9595,Substitution,28.2%,NaN,NaN,NaN,28.2,0.0,3199
47,ORF1ab,L -> K,9601,Substitution,26.4%,NaN,NaN,NaN,26.4,0.0,3201
48,ORF1ab,P -> K,9598,Substitution,26.4% -> 26.9%,NaN,NaN,NaN,26.4,26.9,3200
50,ORF1ab,T -> H,9604,Substitution,26.2% -> 26.4%,NaN,NaN,NaN,26.2,26.4,3202
63,M,Q -> E,55,Substitution,75.7%,NaN,NaN,NaN,75.7,0.0,19
